In [1]:
import tweepy

consumer_key = "fBKRihVu5bNj0ypeey39J8v6x"
consumer_secret = "E4fkBQMCfO9WGg6ZDitWxFRAdWrjIPaL24RTNSpnTXIsfGdO6g"

access_token = "1146388838534660096-iYJJ8NXMf16sNj0O6jd6HzsjvtjfVp"
access_token_secret = "r16edV26xmYuxj1wyzOhgZnvqKcYjnXqxaMRcMv8nUk75"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
print(api)

In [2]:
import time

def process_tweet(tweet) :
    tweet_date = tweet['created_at']
    ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet_date,'%a %b %d %H:%M:%S +0000 %Y'))

    tweet_id = tweet['id']
    tweet_text = tweet['text']
    if('extended_tweet' in tweet.keys()):
        tweet_text = tweet['extended_tweet']['full_text']

    user_id = tweet['user']['id']
    followers_count = tweet['user']['followers_count']
    friends_count = tweet['user']['friends_count']

    user_mentions = tweet['entities']['user_mentions']
    screen_names = [user_mention['screen_name'] for user_mention in user_mentions]
    screen_name = tweet['user']['screen_name']
    retweet_count = tweet['retweet_count']
    favorite_count = tweet['favorite_count']
    retweeted = tweet['retweeted']
    tweet_row = {'date':ts,
                 'tweet_id' : tweet_id,
                 'user_id' : user_id,
                 'followers_count' : followers_count,
                 'friends_count' : friends_count,
                 'user_mentions' : screen_names,
                 'screen_name' : screen_name,
                 'retweet_count' : retweet_count,
                 'favorite_count' : favorite_count,
                 'retweeted' : retweeted,
                 'full_text':tweet_text}
    return tweet_row

In [3]:
#Ref : https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [4]:
#Ref : https://medium.com/@poconnell732/acquiring-free-historical-geo-located-data-from-twitter-1f8f2821e9b1
from TwitterAPI import TwitterAPI
import json
import pandas as pd
from datetime import timedelta, date

api = TwitterAPI(consumer_key=consumer_key,
                 consumer_secret=consumer_secret,
                 access_token_key=access_token,
                 access_token_secret=access_token_secret)
PRODUCT = '30day' #Using 30day API
LABEL = '30daysdev' # sandbox name
web_request_count = 0
                
start_month_date = date(2019, 6, 13)
end_month_date = date(2019, 6, 30)

for date in daterange(start_month_date, end_month_date):
    list_tweets = []
    next_token = {}
    date_str = date.strftime("%Y%m%d")
    start_date = date_str + "0000"
    end_date = date_str + "2359"
    print(start_date)
    print(end_date)
    
    while ((next_token is not None) and (web_request_count<10)):
        print(next_token)
        if not next_token :
            req_dict = {'query' : '@swiggy_in OR @swiggyCares lang:en', 'fromDate': start_date, 'toDate': end_date,}
        else :
            req_dict['next'] = next_token
        
        web_request_count += 1
        print('web_request_count: ', web_request_count)
        req = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL), req_dict)
        print('status_code: ', req.status_code)
        response = req.json()
        if('next' in response.keys()):
            next_token = response['next']
        else :
            next_token = None
        
        #print(response)
        results = response['results']
        with open('swiggy/json/' + date_str + '_' + str(web_request_count) + '.json', 'w+') as f:
            json.dump(results, f)
        
        #print(results)
        for tweet in results:
            tweet_row = process_tweet(tweet)
            list_tweets.append(tweet_row)
        df = pd.DataFrame(list_tweets)
    df.to_csv('swiggy/' + date_str + '.csv', index=False)
    print(df.shape)

201906130000
201906132359
{}
web_request_count:  1
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIzMmM4YmU2ZjM0MTIwZWZhMWJmNjJiMzgyNTNjYmNmMjRhMmE0NmExZDdlZGJhODA3OTM4NmEyOWIwNzUyZmNjIiwiZnJvbURhdGUiOiIyMDE5MDYxMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYxMzIzNTkiLCJuZXh0IjoiMjAxOTA2MTMyMzU5MDAtMTEzOTIyNTQyMTg2NDU2Njc4NC0wIn0=
web_request_count:  2
status_code:  200
eyJhdXRoZW50aWNpdHkiOiJhZjUzNzhkNGZhOTNiM2E2YTBhYjlmM2RmYTcyZjlhOTY5NWY3Y2E0OTc0ZjU3Mzk3Mjg0N2VmNzU4MmRmYWU2IiwiZnJvbURhdGUiOiIyMDE5MDYxMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYxMzIzNTkiLCJuZXh0IjoiMjAxOTA2MTMyMzU5MDAtMTEzOTIwNTQxNjM4NTAxNTgwOC0wIn0=
web_request_count:  3
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI2Njc0Mzc5OGU2NTI0Njk2NjVmZWFmODYzYTM3NWQxZDdmYTRiMDdjMTc3ZDM3YjI2MTcxM2JkNDk2NWI0ZTU5IiwiZnJvbURhdGUiOiIyMDE5MDYxMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYxMzIzNTkiLCJuZXh0IjoiMjAxOTA2MTMyMzU5MDAtMTEzOTE3ODk1MDA1NDEyMTQ3NC0wIn0=
web_request_count:  4
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIyNmMyNzYwMGM3MWI1NmMwZmE4MTk2YjQxZWU4ZDUzMjViNjcyYTMyMGZhZmQ4Yjc5

In [5]:
#Ref : https://medium.com/@poconnell732/acquiring-free-historical-geo-located-data-from-twitter-1f8f2821e9b1
from TwitterAPI import TwitterAPI
import json
import pandas as pd
from datetime import timedelta, date

api = TwitterAPI(consumer_key=consumer_key,
                 consumer_secret=consumer_secret,
                 access_token_key=access_token,
                 access_token_secret=access_token_secret)
PRODUCT = '30day' #Using 30day API
LABEL = '30daysdev' # sandbox name
web_request_count = 0
                
start_month_date = date(2019, 6, 13)
end_month_date = date(2019, 6, 30)

for date in daterange(start_month_date, end_month_date):
    list_tweets = []
    next_token = {}
    date_str = date.strftime("%Y%m%d")
    start_date = date_str + "0000"
    end_date = date_str + "2359"
    print(start_date)
    print(end_date)
    
    while (next_token is not None) and (web_request_count<10):
        print(next_token)
        if not next_token :
            req_dict = {'query' : '@zomatocare OR @zomatoIn lang:en', 'fromDate': start_date, 'toDate': end_date}
        else :
            req_dict['next'] = next_token
        
        web_request_count += 1
        print('web_request_count: ', web_request_count)
        req = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL), req_dict)
        print('status_code: ', req.status_code)
        response = req.json()
        if('next' in response.keys()):
            next_token = response['next']
        else :
            next_token = None
        
        results = response['results']
        with open('zomato/json/' + date_str + '_' + str(web_request_count) + '.json', 'w+') as f:
            json.dump(results, f)
        
        #print(results)
        for tweet in results:
            tweet_row = process_tweet(tweet)
            list_tweets.append(tweet_row)
        df = pd.DataFrame(list_tweets)
    df.to_csv('zomato/' + date_str + '.csv', index=False)
    print(df.shape)

201906130000
201906132359
{}
web_request_count:  1
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI1ZjdmYTE2OTViNTI2ODVhMjUzOGFhMGU5ZTRmZWVjMjBhYmQ4ODc5YmZjZTQ5ZGViNmUwNTkxNTY2MjViNWRhIiwiZnJvbURhdGUiOiIyMDE5MDYxMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYxMzIzNTkiLCJuZXh0IjoiMjAxOTA2MTMyMzU5MDAtMTEzOTIzNDg0OTk4MDUwMjAxNi0wIn0=
web_request_count:  2
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIzNDIwYjkyMGM0MGY5ZDk1NDM0NjE3M2VhNGM5NmRkZmIwNTRhZmM3NDQxYjBhYzM0ODUzZjUyYzIyZmY2NTdjIiwiZnJvbURhdGUiOiIyMDE5MDYxMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYxMzIzNTkiLCJuZXh0IjoiMjAxOTA2MTMyMzU5MDAtMTEzOTIxOTE2Mjg2ODI2OTA1Ni0wIn0=
web_request_count:  3
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI5YjMzNWZlN2FhZWJmZjRhOTkyMDc1NDA0Mzc3ZWY3OTkyYTc5ZGY0NjRmNGNhMGY4NDI0NjExNGIxZGRmZjAzIiwiZnJvbURhdGUiOiIyMDE5MDYxMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYxMzIzNTkiLCJuZXh0IjoiMjAxOTA2MTMyMzU5MDAtMTEzOTIwNDQ2OTUzNzcyMjM2OC0wIn0=
web_request_count:  4
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIwMGE0OWQ4ZTU3N2Y4OTRiZmFlM2Q4MGM4ZDMxYzM4YzFjNGYyNTNlNDYxNzA1MDE3